In [ ]:
!pip install rich
!pip install git+https://github.com/JR-1991/sdrdm-database.git

In [12]:
import rich

from sdRDM import DataModel
from sdrdm_database import DBConnector
from sdrdm_database.dbconnector import SupportedBackends

# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype=SupportedBackends.MYSQL
)

🎉 Connected                                                                                        


In [3]:
# Check, if the tables have been created!
db.connection.list_tables()

['Activation',
 'AssayConditions',
 'Author',
 'CompoundClassification',
 'Dataset',
 'Dataset_assay_conditions_AssayConditions',
 'Dataset_results_set_ResultsSet',
 'Experiment',
 'Inhibition',
 'InitialKineticsParameters',
 'MacromolecularComponents',
 'PosttranslationalModifications',
 'ProteinDescription',
 'ProteinDescription_protein_characterization_Dataset',
 'ProteinReaction',
 'ProteinSource',
 'Publication',
 'Publication_author_Author',
 'Publication_experiment_Experiment',
 'ResultsSet',
 'RoleOfComponent',
 'SequenceModifications',
 'SmallAssayComponents',
 '__model_meta__']

In [14]:
# Grab the "Publication" model from the Database
Publication = db.get_table_api("Publication")

# Build dataset to insert later!
dataset = Publication(title="Test", doi="SomeDOI", pmid="SomePMID")
dataset.add_to_author(name="StrendaMan", affiliation="Strenda")

print(dataset.tree())

Publication
├── id [value=4c63272b-e99e-4e05-aa09-cf751fa4d62b]
├── title [value=Test]
├── doi [value=SomeDOI]
├── pmid [value=SomePMID]
└── author [value=None]
    └── 0
        └── Author
            ├── id [value=02e113da-37b1-486f-8dfa-231ffcf11940]
            ├── name [value=StrendaMan]
            └── affiliation [value=Strenda]
ClassNode(/Publication, class_name=None, constants={}, id=None, module=None, outer_type=None)


In [6]:
# Insert the row into the database
db.insert(dataset)

✅ Inserted 1 rows into the database.

In [7]:
# Let's create more random datasets
import random

options = ["Strenda", "Biocatalysis", "EnzymeML", "Workshop"] 
datasets = []

for _ in range(20):
    dataset = libStrenda.Publication(
        title=random.choice(options),
        doi=random.choice(options),
        pmid=random.choice(options),
    )
    
    dataset.add_to_author(name="StrendaMan", affiliation="Strenda")

    datasets.append(dataset)

# Add all of them at once!
db.insert(*datasets)

✅ Inserted 20 rows into the database.

In [16]:
# Lets fetch them from the database!
result = db.get(db.get_table_api("Publication"))

# Should be 21! Lets inspect the amount and the first entry
rich.print(f"{len(result)} entries found!")
rich.print(result[0].tree())

21 entries found!

Publication
├── id [value=00c65cff-21d0-493f-81aa-5d32d8992474]
├── title [value=Test]
├── doi [value=SomeDOI]
├── pmid [value=SomePMID]
└── author [value=None]
    └── 0
        └── Author
            ├── id [value=6bf4f0db-82eb-431e-9eef-139a83149bca]
            ├── name [value=StrendaMan]
            └── affiliation [value=Strenda]


ClassNode(/Publication, class_name=None, constants={}, id=None, module=None, outer_type=None)

In [17]:
# You can also view individual tables
db.connection.table("Publication")

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ title        ┃ doi          ┃ pmid         ┃ id                                   ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string      │ string       │ string       │ !string                              │
├──────────────┼──────────────┼──────────────┼──────────────────────────────────────┤
│ Test         │ SomeDOI      │ SomePMID     │ 00c65cff-21d0-493f-81aa-5d32d8992474 │
│ Workshop     │ Biocatalysis │ Biocatalysis │ 0b345fa6-7b82-4544-ac66-f08f3e91cfee │
│ Workshop     │ Biocatalysis │ Workshop     │ 0f25b517-1a5e-4da8-8dc9-7b5a87658832 │
│ EnzymeML     │ Workshop     │ Strenda      │ 20ac6092-8697-4714-a120-9e476d3a613f │
│ Workshop     │ Strenda      │ Workshop     │ 267b9093-0252-47b8-96f2-5fae11398c44 │
│ Workshop     │ Strenda      │ Strenda      │ 289c4ab5-1677-4c7b-886a-d84b88b164a3 │
│ Workshop     │ EnzymeML     │ EnzymeML     │ 3b63d1dc-a6bd-4fb1-a65a-ae6af97712cc │
│ Biocatalysis │ Biocatalysis │ EnzymeML     │ 3da1b160-5e29-47bb-9912-3d32193b2486 │
│ Strenda      │ Biocatalysis │ Workshop     │ 44f9a8af-c387-4a65-9761-de63ffbc75e0 │
│ Strenda      │ Strenda      │ Biocatalysis │ 4d108d69-4f02-4ada-a506-52198f7e64c7 │
│ …            │ …            │ …            │ …                                    │
└──────────────┴──────────────┴──────────────┴──────────────────────────────────────┘

In [19]:
# Suppose, we only want to add an author. You can proceed in the same way as above.
Author = db.get_table_api("Author")
author1 = Author(name="John Doe")

# Add and lets check the table
db.insert(author1)
table = db.connection.table("Author")
print(table)

# We can also filter the table
rich.print("Looking for John Doe")
table[table.name == "John Doe"]

✅ Inserted 1 rows into the database.

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name       ┃ affiliation ┃ email  ┃ orcid  ┃ id                                   ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string    │ string      │ string │ string │ !string                              │
├────────────┼─────────────┼────────┼────────┼──────────────────────────────────────┤
│ StrendaMan │ Strenda     │ NULL   │ NULL   │ 00594dae-d6e0-4b6e-812f-0b994359f4af │
│ StrendaMan │ Strenda     │ NULL   │ NULL   │ 1c60c513-a4e8-49c3-beb5-a07fa5022574 │
│ StrendaMan │ Strenda     │ NULL   │ NULL   │ 261012b1-446d-48da-8d70-8225256a497b │
│ StrendaMan │ Strenda     │ NULL   │ NULL   │ 33cce500-64de-4e8a-b64d-3f78c30a8367 │
│ StrendaMan │ Strenda     │ NULL   │ NULL   │ 3e28623f-c5e7-4887-8025-bcd8f21afd29 │
│ StrendaMan │ Strenda     │ NULL   │ NULL   │ 4362f522-28ad-4a73-95a8-171f24f2c4b3 │
│ StrendaMan │ Strenda     │ NULL   │ NULL   │ 461f8c9

Looking for John Doe

┏━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name     ┃ affiliation ┃ email  ┃ orcid  ┃ id                                   ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string  │ string      │ string │ string │ !string                              │
├──────────┼─────────────┼────────┼────────┼──────────────────────────────────────┤
│ John Doe │ NULL        │ NULL   │ NULL   │ 58c672ad-962e-45df-b820-e384003455b7 │
│ John Doe │ NULL        │ NULL   │ NULL   │ 5e407774-8196-4800-bb5f-ea9e1633c85e │
└──────────┴─────────────┴────────┴────────┴──────────────────────────────────────┘